# Question and Answer Chat Bots

## Loading the Data

We will be working with the Babi Data Set from Facebook Research.

Full Details: https://research.fb.com/downloads/babi/

- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698


In [1]:
import pickle
import numpy as np

In [ ]:
with open("train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp)
with open("test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

## Exploring the Format of the Data

In [4]:
type(test_data)

list

In [5]:
type(train_data)

list

In [6]:
len(test_data)

1000

In [7]:
len(train_data)

10000

In [8]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [9]:
# first story
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [10]:
# question
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [11]:
# answer
train_data[0][2]

'no'

## Setting up Vocabulary of All Words

In [12]:
# Create a set that holds the vocab words
vocab = set()
all_data = test_data + train_data
for story, question , answer in all_data:
    # In case you don't know what a union of sets is:
    # https://www.programiz.com/python-programming/methods/set/union
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [15]:
vocab.add('no')
vocab.add('yes')

In [16]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [ ]:
vocab_len = len(vocab) + 1 # we add an extra space to hold a 0 for Keras's pad_sequences
max_story_len = max([len(data[0]) for data in all_data])

In [19]:
max_story_len

156

In [21]:
max_question_len = max([len(data[1]) for data in all_data])
max_question_len

6

## Vectorizing the Data

In [22]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [23]:
# Reserve 0 for pad_sequences
vocab_size = len(vocab) + 1

In [24]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [25]:
# integer encode sequences of words
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [26]:
tokenizer.word_index

{'journeyed': 1,
 'sandra': 2,
 'office': 3,
 'apple': 4,
 'dropped': 5,
 'daniel': 6,
 '.': 7,
 'to': 8,
 'mary': 9,
 'john': 10,
 'put': 11,
 'hallway': 12,
 'got': 13,
 'went': 14,
 '?': 15,
 'bathroom': 16,
 'back': 17,
 'up': 18,
 'took': 19,
 'kitchen': 20,
 'is': 21,
 'down': 22,
 'milk': 23,
 'yes': 24,
 'grabbed': 25,
 'there': 26,
 'in': 27,
 'left': 28,
 'no': 29,
 'picked': 30,
 'bedroom': 31,
 'discarded': 32,
 'football': 33,
 'garden': 34,
 'the': 35,
 'moved': 36,
 'travelled': 37}

In [27]:
train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answer_text.append(answer)

In [28]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [29]:
len(train_story_text)

10000

In [30]:
len(train_story_seq)

10000

In [31]:
# word_index = tokenizer.word_index

### Functionalize Vectorization

In [32]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len, max_question_len=max_question_len):
    '''
    INPUT:     
    data: consisting of Stories,Queries,and Answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story (used for pad_sequences function)
    max_question_len: length of the longest question (used for pad_sequences function)

    OUTPUT:    
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    
    X = []        # X = STORIES
    Xq = []       # Xq = QUERY/QUESTION
    Y = []        # Y = CORRECT ANSWER
    
    for story, query, answer in data:    
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(len(word_index) + 1)
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [33]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [34]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [35]:
inputs_test

array([[ 0,  0,  0, ..., 35, 31,  7],
       [ 0,  0,  0, ..., 35, 34,  7],
       [ 0,  0,  0, ..., 35, 34,  7],
       ...,
       [ 0,  0,  0, ..., 35,  4,  7],
       [ 0,  0,  0, ..., 35, 34,  7],
       [ 0,  0,  0, ...,  4, 26,  7]], dtype=int32)

In [36]:
queries_test

array([[21, 10, 27, 35, 20, 15],
       [21, 10, 27, 35, 20, 15],
       [21, 10, 27, 35, 34, 15],
       ...,
       [21,  9, 27, 35, 31, 15],
       [21,  2, 27, 35, 34, 15],
       [21,  9, 27, 35, 34, 15]], dtype=int32)

In [37]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0., 497.,   0.,   0.,   0.,   0., 503.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

In [39]:
tokenizer.word_index['yes']

24

In [40]:
tokenizer.word_index['no']

29

## Creating the Model

In [41]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

### Placeholders for Inputs

Recall we technically have two inputs, stories and questions. So we need to use placeholders. `Input()` is used to instantiate a Keras tensor.


In [42]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

### Building the Networks

To understand why we chose this setup, make sure to read the paper we are using:

* Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, Rob Fergus,
  "End-To-End Memory Networks",
  http://arxiv.org/abs/1503.08895

## Encoders

### Input Encoder m

In [43]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

Instructions for updating:
Colocations handled automatically by placer.


### Input Encoder c

In [44]:
# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

### Question Encoder

In [45]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=max_question_len))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

### Encode the Sequences

In [46]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

##### Use dot product to compute the match between first input vector seq and the query

In [47]:
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

#### Add this match matrix with the second input vector sequence

In [48]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

#### Concatenate

In [49]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

In [50]:
answer

<tf.Tensor 'concatenate_1/concat:0' shape=(?, 6, 220) dtype=float32>

In [51]:
# Reduce with RNN (LSTM)
answer = LSTM(32)(answer)  # (samples, 32)

OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-11
OMP: Info #217: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #157: KMP_AFFINITY: 12 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #288: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #288: KMP_AFFINITY: topology layer "L3 cache" is equivalent to "socket".
OMP: Info #288: KMP_AFFINITY: topology layer "L2 cache" is equivalent to "core".
OMP: Info #288: KMP_AFFINITY: topology layer "L1 cache" is equivalent to "core".
OMP: Info #192: KMP_AFFINITY: 1 socket x 6 cores/socket x 2 threads/core (6 total cores)
OMP: Info #219: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 0 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 2 maps to socket 0 core 1 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 3 maps to socket 0 core 1 thread 1 
OMP: Info

In [52]:
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

In [53]:
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [54]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 156)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
____________________________________________________________________________________________

In [55]:
# train
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=120,validation_data=([inputs_test, queries_test], answers_test))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


2022-08-22 03:45:51.835978: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-08-22 03:45:51.889417: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2592005000 Hz
2022-08-22 03:45:51.890164: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55d55bd52540 executing computations on platform Host. Devices:
2022-08-22 03:45:51.890250: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-08-22 03:45:51.904026: I tensorflow/core/common_runtime/process_util.cc:71] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Train on 10000 samples, validate on 1000 samples
Epoch 1/120


OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3740 thread 6 bound to OS proc set 1
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3765 thread 7 bound to OS proc set 3
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3766 thread 8 bound to OS proc set 5
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3767 thread 9 bound to OS proc set 7
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3768 thread 10 bound to OS proc set 9
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3769 thread 11 bound to OS proc set 11
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3770 thread 12 bound to OS proc set 0
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3739 thread 13 bound to OS proc set 2
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3772 thread 15 bound to OS proc set 6
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3773 thread 16 bound to OS proc set 8
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3771 thread 14 bound to OS proc set 4
OMP: Info #255: KMP_AFFINITY: pid 3712 tid 3774 thread 17 bound to OS proc set 10
OMP: Info #255: KMP_AFFINITY: 

10000/10000 [==============================] - 13s 1ms/step - loss: 0.9796 - accuracy: 0.4922 - val_loss: 0.7010 - val_accuracy: 0.4970
Epoch 2/120
10000/10000 [==============================] - 11s 1ms/step - loss: 0.7098 - accuracy: 0.4978 - val_loss: 0.6946 - val_accuracy: 0.4970
Epoch 3/120
10000/10000 [==============================] - 11s 1ms/step - loss: 0.6971 - accuracy: 0.5004 - val_loss: 0.6935 - val_accuracy: 0.5030
Epoch 4/120
10000/10000 [==============================] - 11s 1ms/step - loss: 0.6954 - accuracy: 0.4982 - val_loss: 0.6940 - val_accuracy: 0.5030
Epoch 5/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.6949 - accuracy: 0.4984 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 6/120
10000/10000 [==============================] - 10s 960us/step - loss: 0.6940 - accuracy: 0.5065 - val_loss: 0.6942 - val_accuracy: 0.4970
Epoch 7/120
10000/10000 [==============================] - 10s 966us/step - loss: 0.6945 - accuracy: 0.4981 - val_loss: 0.

Epoch 56/120
10000/10000 [==============================] - 10s 968us/step - loss: 0.3189 - accuracy: 0.8630 - val_loss: 0.4154 - val_accuracy: 0.8180
Epoch 57/120
10000/10000 [==============================] - 10s 958us/step - loss: 0.3197 - accuracy: 0.8637 - val_loss: 0.3976 - val_accuracy: 0.8160
Epoch 58/120
10000/10000 [==============================] - 10s 954us/step - loss: 0.3141 - accuracy: 0.8697 - val_loss: 0.4016 - val_accuracy: 0.8200
Epoch 59/120
10000/10000 [==============================] - 11s 1ms/step - loss: 0.3121 - accuracy: 0.8698 - val_loss: 0.4398 - val_accuracy: 0.8130
Epoch 60/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.3100 - accuracy: 0.8686 - val_loss: 0.4230 - val_accuracy: 0.8130
Epoch 61/120
10000/10000 [==============================] - 11s 1ms/step - loss: 0.3138 - accuracy: 0.8676 - val_loss: 0.3985 - val_accuracy: 0.8230
Epoch 62/120
10000/10000 [==============================] - 10s 982us/step - loss: 0.3082 - accuracy

10000/10000 [==============================] - 10s 961us/step - loss: 0.2317 - accuracy: 0.9046 - val_loss: 0.5787 - val_accuracy: 0.7910
Epoch 111/120
10000/10000 [==============================] - 10s 958us/step - loss: 0.2264 - accuracy: 0.9071 - val_loss: 0.5680 - val_accuracy: 0.7880
Epoch 112/120
10000/10000 [==============================] - 12s 1ms/step - loss: 0.2295 - accuracy: 0.9067 - val_loss: 0.5203 - val_accuracy: 0.8030
Epoch 113/120
10000/10000 [==============================] - 10s 961us/step - loss: 0.2254 - accuracy: 0.9097 - val_loss: 0.5978 - val_accuracy: 0.8010
Epoch 114/120
10000/10000 [==============================] - 10s 1ms/step - loss: 0.2324 - accuracy: 0.9045 - val_loss: 0.5540 - val_accuracy: 0.7980
Epoch 115/120
10000/10000 [==============================] - 11s 1ms/step - loss: 0.2209 - accuracy: 0.9095 - val_loss: 0.6022 - val_accuracy: 0.8100
Epoch 116/120
10000/10000 [==============================] - 9s 936us/step - loss: 0.2229 - accuracy: 0.9108

### Saving the Model

In [56]:
filename = 'chatbot_120_epochs.h5'
model.save(filename)

## Evaluating the Model

### Plotting Out Training History

In [57]:
import matplotlib.pyplot as plt
%matplotlib inline
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy'])


KeyError: 'acc'

### Evaluating on Given Test Set

In [ ]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

In [ ]:
test_data[0][0]

In [ ]:
story =' '.join(word for word in test_data[0][0])
print(story)

In [ ]:
query = ' '.join(word for word in test_data[0][1])
print(query)

In [ ]:
print("True Test Answer from Data is:",test_data[0][2])

In [ ]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

## Writing Your Own Stories and Questions

Remember you can only use words from the existing vocab

In [ ]:
vocab

In [ ]:
# Note the whitespace of the periods
my_story = "John left the kitchen . Sandra dropped the football in the garden ."
my_story.split()

In [ ]:
my_question = "Is the football in the garden ?"

In [ ]:
my_question.split()

In [ ]:
mydata = [(my_story.split(),my_question.split(),'yes')]

In [ ]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [ ]:
pred_results = model.predict(([ my_story, my_ques]))

In [ ]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])